# Data Retrieval

<br>

### Imports

In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import io
import requests

from IPython.display import HTML

<br>

### Foreign Exchange & GDP

US/EU

In [3]:
def query_FX(currency_from:str, currency_to:str, end_date:str, start_date='1999-01-01'):
    stream = f'EX{currency_from}{currency_to}'
    
    stlouisfed_url = 'https://fred.stlouisfed.org/graph/fredgraph.csv'

    params = {
        'id': stream,
        'cosd': start_date,
        'coed': end_date,
    }

    headers = {
        'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36',
    }

    r = requests.get(stlouisfed_url, params=params, headers=headers)

    df = pd.read_csv(io.StringIO(r.content.decode('utf-8'))).sort_values('DATE', ascending=False)

    return df

end_date = '2020-06-25'
currency_from = 'US'
currency_to = 'EU'

df = query_FX(currency_from, currency_to, end_date)

df.head()

DATE  EXUSEU
256  2020-05-01  1.0907
255  2020-04-01  1.0871
254  2020-03-01  1.1046
253  2020-02-01  1.0911
252  2020-01-01  1.1098

<br>

US/UK

In [4]:
currency_from = 'US'
currency_to = 'UK'

df = query_FX(currency_from, currency_to, end_date)

df.head()

DATE  EXUSUK
256  2020-05-01  1.2302
255  2020-04-01  1.2420
254  2020-03-01  1.2369
253  2020-02-01  1.2953
252  2020-01-01  1.3076

<br>

CA/US

In [5]:
currency_from = 'CA'
currency_to = 'US'

df = query_FX(currency_from, currency_to, end_date)

df.head()

DATE  EXCAUS
256  2020-05-01  1.3972
255  2020-04-01  1.4048
254  2020-03-01  1.3960
253  2020-02-01  1.3286
252  2020-01-01  1.3089

<br>

US/AL

In [6]:
currency_from = 'US'
currency_to = 'AL'

df = query_FX(currency_from, currency_to, end_date)

df.head()

DATE  EXUSAL
256  2020-05-01  0.6517
255  2020-04-01  0.6312
254  2020-03-01  0.6218
253  2020-02-01  0.6664
252  2020-01-01  0.6851

<br>

US GDP

In [7]:
def query_GDP(end_date:str, start_date='1999-01-01'):
    stream = f'GDPDEF'
    
    stlouisfed_url = 'https://fred.stlouisfed.org/graph/fredgraph.csv'

    params = {
        'id': stream,
        'cosd': start_date,
        'coed': end_date,
    }

    headers = {
        'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36',
    }

    r = requests.get(stlouisfed_url, params=params, headers=headers)

    df = pd.read_csv(io.StringIO(r.content.decode('utf-8'))).sort_values('DATE', ascending=False)

    return df

query_GDP(end_date)

DATE   GDPDEF
84  2020-01-01  113.493
83  2019-10-01  113.043
82  2019-07-01  112.664
81  2019-04-01  112.188
80  2019-01-01  111.473
..         ...      ...
4   2000-01-01   77.396
3   1999-10-01   76.873
2   1999-07-01   76.462
1   1999-04-01   76.201
0   1999-01-01   75.926

[85 rows x 2 columns]

<br>

### Alberta Hub Price

In [9]:
alberta_url = 'https://www.alberta.ca/alberta-natural-gas-reference-price.aspx'

df = (pd
      .read_html(alberta_url)
      [1]
      .set_index('Unnamed: 0')
     )

df = df.unstack().reset_index()
df.columns = ['month', 'year', 'price']
df.index = pd.to_datetime(df['year'].astype(str) + ' ' + df['month'].astype(str), format='%Y %B')
s_price = df.sort_index(ascending=False)['price'].dropna()

s_price

2020-04-01    1.56
2020-03-01    1.60
2020-02-01    1.79
2020-01-01    2.06
2019-12-01    2.22
              ... 
1994-05-01    1.80
1994-04-01    1.82
1994-03-01    2.07
1994-02-01    2.04
1994-01-01    1.93
Name: price, Length: 316, dtype: float64

<br>

### Victoria Hub Price

In [10]:
csv_url = 'http://www.nemweb.com.au/REPORTS/CURRENT/VicGas/INT310_V4_PRICE_AND_WITHDRAWALS_1.CSV'

df = pd.read_csv(csv_url)
s = df['price_value'].groupby(pd.to_datetime(df['gas_date'])).mean()

s.resample('m').mean().round(2).sort_index(ascending=False)

gas_date
2020-06-30    4.59
2020-05-31    4.67
2020-04-30    4.54
2020-03-31    4.86
2020-02-29    5.87
2020-01-31    6.28
2019-12-31    6.42
2019-11-30    7.28
2019-10-31    8.49
2019-09-30    8.28
2019-08-31    8.42
2019-07-31    8.32
2019-06-30    9.41
Freq: -1M, Name: price_value, dtype: float64